In [ ]:
!pip install qiskit
!pip install qiskit-machine-learning

In [1]:
from IPython.display import clear_output
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp
from qiskit.utils import algorithm_globals
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from sklearn.model_selection import train_test_split
import torch
from torch.optim import SGD


In [2]:
algorithm_globals.random_seed = 42

## Define the quantum circuit

In [3]:
n_qubits = 4

In [4]:
def layer(W):
    """Applies a layer of arbitrary rotations and circular entanglements to the variational circuit

    Args:
        W (np.ndarray): rotation parameters for the layer
    """
    qc = QuantumCircuit(n_qubits)

    for i in range(n_qubits):
        qc.rz(W[i*3], i)
        qc.ry(W[i*3 + 1], i)
        qc.rz(W[i*3 + 2], i)

    for i in range(n_qubits-1):
        qc.cnot(i, i+1)

    if n_qubits > 2:
        qc.cnot(n_qubits-1, 0)

    return qc

In Qiskit, there is not basis state preparation circuit and anyway we need to work with `Parameter` objects to use the `qiskit-machine-learning` tools. So we just encode $R_Y$ rotation, pass the parameters and multiply by $\pi$.

In [34]:
def global_phase_gate(qc, phase, qubit):
    qc.p(phase, qubit)
    qc.x(qubit)
    qc.p(phase, qubit)
    qc.x(qubit)


def statepreparation(x):
    """Prepares the binary state fed to the vqc

    Args:
        x (List): list of 0s and 1s corresponding to the basis state
    """
    qc = QuantumCircuit(n_qubits)
    
    for i, x_i in enumerate(x):
        qc.rx(x_i * np.pi, i)
        global_phase_gate(qc, -np.pi/2, i)


    return qc

In [35]:
n_layers = 2

weight_params = ParameterVector(name='W', length=3 * n_qubits * n_layers)
input_params = ParameterVector(name='x', length=n_qubits)

In [36]:
# prepare the quantum circuit
qc = QuantumCircuit(n_qubits)
qc = qc.compose(statepreparation(input_params))
for l in range(n_layers):
    qc = qc.compose(layer(weight_params[3 * n_qubits * l: 3 * n_qubits * (l+1)]))


In [37]:
qc.draw()

┌────────────┐┌─────────┐┌───┐┌─────────┐┌───┐┌──────────┐ ┌──────────┐»
q_0: ┤ Rx(π*x[0]) ├┤ P(-π/2) ├┤ X ├┤ P(-π/2) ├┤ X ├┤ Rz(W[0]) ├─┤ Ry(W[1]) ├»
     ├────────────┤├─────────┤├───┤├─────────┤├───┤├──────────┤ ├──────────┤»
q_1: ┤ Rx(π*x[1]) ├┤ P(-π/2) ├┤ X ├┤ P(-π/2) ├┤ X ├┤ Rz(W[3]) ├─┤ Ry(W[4]) ├»
     ├────────────┤├─────────┤├───┤├─────────┤├───┤├──────────┤ ├──────────┤»
q_2: ┤ Rx(π*x[2]) ├┤ P(-π/2) ├┤ X ├┤ P(-π/2) ├┤ X ├┤ Rz(W[6]) ├─┤ Ry(W[7]) ├»
     ├────────────┤├─────────┤├───┤├─────────┤├───┤├──────────┤┌┴──────────┤»
q_3: ┤ Rx(π*x[3]) ├┤ P(-π/2) ├┤ X ├┤ P(-π/2) ├┤ X ├┤ Rz(W[9]) ├┤ Ry(W[10]) ├»
     └────────────┘└─────────┘└───┘└─────────┘└───┘└──────────┘└───────────┘»
«      ┌──────────┐                       ┌───┐┌───────────┐┌───────────┐»
«q_0: ─┤ Rz(W[2]) ├──■────────────────────┤ X ├┤ Rz(W[12]) ├┤ Ry(W[13]) ├»
«      ├──────────┤┌─┴─┐     ┌───────────┐└─┬─┘├───────────┤├───────────┤»
«q_1: ─┤ Rz(W[5]) ├┤ X ├──■──┤ Rz(W[15]) ├──┼──┤ Ry(W[16]) ├┤ Rz(W[17]) ├»
«      ├──────────┤└───┘┌─┴─┐└───────────┘  │  ├───────────┤├───────────┤»
«q_2: ─┤ Rz(W[8]) ├─────┤ X ├──────■────────┼──┤ Rz(W[18]) ├┤ Ry(W[19]) ├»
«     ┌┴──────────┤     └───┘    ┌─┴─┐      │  ├───────────┤├───────────┤»
«q_3: ┤ Rz(W[11]) ├──────────────┤ X ├──────■──┤ Rz(W[21]) ├┤ Ry(W[22]) ├»
«     └───────────┘              └───┘         └───────────┘└───────────┘»
«     ┌───────────┐               ┌───┐
«q_0: ┤ Rz(W[14]) ├──■────────────┤ X ├
«     └───────────┘┌─┴─┐          └─┬─┘
«q_1: ─────────────┤ X ├──■─────────┼──
«     ┌───────────┐└───┘┌─┴─┐       │  
«q_2: ┤ Rz(W[20]) ├─────┤ X ├──■────┼──
«     ├───────────┤     └───┘┌─┴─┐  │  
«q_3: ┤ Rz(W[23]) ├──────────┤ X ├──■──
«     └───────────┘          └───┘

In [38]:
# define an observable
observable = SparsePauliOp.from_list([("IIIZ", 1)])

In [39]:
# define the qnn
qnn = EstimatorQNN(circuit=qc, observables=observable, input_params=input_params, weight_params=weight_params, input_gradients=True)  # now qnn is part of a hybrid computational graph: we should set `input_gradients=True`

In [40]:
class QnnWBias(torch.nn.Module):

    def __init__(self, qnn, qnn_weights_init) -> None:
        super().__init__()

        self.qnn = TorchConnector(qnn, initial_weights=qnn_weights_init)
        self.linear = torch.nn.Linear(1, 1)

    def forward(self, x):
        x = self.qnn(x)
        return self.linear(x)
        # return x

In [41]:
input_trial = torch.Tensor([[1, 1, 0, 0]])
# qnn_weights_init = .01 *  algorithm_globals.random.normal(size=qnn.num_weights)
qnn_weights_init = np.pi * algorithm_globals.random.random(size=qnn.num_weights)

In [42]:
qnn_weights_init

array([1.13666784, 0.27536034, 0.37072648, 3.02189064, 2.85439042,
       2.19819479, 0.83525512, 3.04475739, 2.44651812, 2.25217695,
       1.41171079, 0.85527209, 0.30282114, 2.83560906, 1.43186344,
       0.63574326, 0.96119108, 1.81967194, 0.55534808, 2.69113314,
       2.38295938, 2.26025954, 1.35746032, 1.97074885])

In [27]:
model = QnnWBias(qnn=qnn, qnn_weights_init=qnn_weights_init)

In [28]:
model(torch.Tensor([0, 0, 1, 1]))

tensor([-1.0132], grad_fn=<AddBackward0>)

## Data
### Load, preprocess and split

In [29]:
data = np.loadtxt("data/parity.txt")

X = np.array(data[:, :-1])
y = np.array(data[:, -1])

# shift lables from [0, 1] to [-1, 1], to match the range of expectation values
y = 2 * y - np.ones(len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=algorithm_globals.random_seed, shuffle=True)

In [30]:
# convert to tensors
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)

In [31]:
model(X_train)

tensor([[-1.2892],
        [-0.9926],
        [-1.0385],
        [-1.0132],
        [-1.0385],
        [-1.0974],
        [-1.2769],
        [-1.1226],
        [-1.0283],
        [-1.3260],
        [-1.0132],
        [-1.1767]], grad_fn=<AddmmBackward0>)

## Train the model

In [34]:
# first we define the accuracy function

def accuracy(labels, predictions):
    """Returns the accuracy over the dataset

    Args:
        labels (torch.Tensor): true values
        predictions (torch.Tensor): model-predicted values

    Returns:
        float: number of accurate predictions over total
    """
    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss += 1

    loss /= len(labels)
    return loss

In [35]:
loss_func = torch.nn.MSELoss()
opt = SGD(params=model.parameters(), lr=0.5, momentum=0.9, nesterov=True)

iterations = 50
batch_size = 5

In [36]:
for it in range(iterations):

    # shuffle the batch indices
    batch_index = algorithm_globals.random.integers(0, len(X_train), size=batch_size)

    X_batch = X_train[batch_index]
    y_batch = y_train[batch_index]

    opt.zero_grad(set_to_none=True)
    output = model(X_batch)
    loss = loss_func(output, y_batch)
    loss.backward()  # computes the gradient of the loss
    opt.step()

    with torch.no_grad():
        # compute loss over train dataset
        loss_train = loss_func(model(X_train), y_train)
        # compute accuracy over train dataset
        predictions = np.sign(model(X_train))
        acc = accuracy(y_train, predictions)

    print(
        "Iter: {:5d} | Loss: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, loss_train, acc
        )
    )

/home/giorgio/Phd/qnn-demo-2023/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/giorgio/Phd/qnn-demo-2023/venv/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Iter:     1 | Loss: 4.5337868 | Accuracy: 0.5000000 
Iter:     2 | Loss: 1.0111471 | Accuracy: 0.5000000 
Iter:     3 | Loss: 1.5008314 | Accuracy: 0.5000000 
Iter:     4 | Loss: 2.2876463 | Accuracy: 0.5000000 
Iter:     5 | Loss: 1.1834501 | Accuracy: 0.5000000 
Iter:     6 | Loss: 2.1430247 | Accuracy: 0.5000000 
Iter:     7 | Loss: 6.3119755 | Accuracy: 0.5000000 
Iter:     8 | Loss: 15.2277870 | Accuracy: 0.5000000 
Iter:     9 | Loss: 5.4348764 | Accuracy: 0.5000000 
Iter:    10 | Loss: 1.3491863 | Accuracy: 0.4166667 
Iter:    11 | Loss: 1.4604230 | Accuracy: 0.4166667 
Iter:    12 | Loss: 1.6295033 | Accuracy: 0.5000000 
Iter:    13 | Loss: 1.3837266 | Accuracy: 0.5000000 
Iter:    14 | Loss: 1.8131251 | Accuracy: 0.5000000 
Iter:    15 | Loss: 1.4316691 | Accuracy: 0.5833333 
Iter:    16 | Loss: 6.2255259 | Accuracy: 0.5000000 
Iter:    17 | Loss: 3.5554576 | Accuracy: 0.5000000 
Iter:    18 | Loss: 2.5925612 | Accuracy: 0.5000000 
Iter:    19 | Loss: 4.8871431 | Accuracy: 0.5

In [ ]:
model(X_train)